In [ ]:
####----------------------点位不正确，需要检查，有重复点
Candidate_Distor_Points = 10 ; scale=1
pointList = ee.List(Points_WithH_Angle.get(0))
index = ee.List(unionFilter_indices.get(0)).get(0)

# 获取极值点
extreme_point = ee.Dictionary(pointList.get(index))

# 获取极值点前十位数的索引范围及前十位数的点
start_index_left = ee.Number(index).subtract(Candidate_Distor_Points).max(0)  # 确保索引不小于 0
end_index_left = index
previous_points_left = pointList.slice(start_index_left, end_index_left)

start_index_Right = ee.Number(index).add(Candidate_Distor_Points).min(pointList.size().subtract(1))  # 确保索引不大于 pointList 的最大索引
end_index_Right = index
afters_points_Right = pointList.slice(ee.Number(end_index_Right).add(1),start_index_Right.add(1)).reverse()

# ----------------------------------------Left layover----------------------------------------
features_Left = calculate_Left_Layover(previous_points_left,extreme_point, scale)

# ----------------------------------------Right layover----------------------------------------
## 当存在Left layover才执行Right layover判断
distorted_features_Left = ee.FeatureCollection(features_Left).filter(ee.Filter.eq('Distortion', 1))
features_Right = ee.Algorithms.If(
    distorted_features_Left.size().gt(0),
    calculate_Right_Layover(distorted_features_Left,extreme_point, afters_points_Right, scale),
    ee.List([])
)

# ----------------------------------------Shadow----------------------------------------
Shadow = calculate_Shadow(afters_points_Right,extreme_point,scale)
Distor_featureCollection = ee.FeatureCollection(ee.List([features_Left,features_Right,Shadow]).flatten())

Leftlayover  = Distor_featureCollection.filter(ee.Filter.And(
                    ee.Filter.eq('Distortion', 1),
                    ee.Filter.eq('Distortion Type', 'Leftlayover')))
Rightlayover = Distor_featureCollection.filter(ee.Filter.And(
                    ee.Filter.eq('Distortion', 1),
                    ee.Filter.eq('Distortion Type', 'Rightlayover')))
Shadow       = Distor_featureCollection.filter(ee.Filter.And(
                    ee.Filter.eq('Distortion', 1),
                    ee.Filter.eq('Distortion Type', 'Shadow')))

In [ ]:
Map = geemap.Map()
Map.centerObject(AOI, zoom=15)
Map.addLayer(Leftlayover, {'color': 'red'}, 'Leftlayover')
Map.addLayer(Rightlayover, {'color': 'blue'}, 'Rightlayover')
Map.addLayer(Shadow, {'color': 'green'}, 'Shadow')
Map.addLayer(ee.Geometry.Point(ee.List(ee.Dictionary(extreme_point).get('point_coordinates'))), {'color': 'black'}, 'extreme_point')
Map

In [ ]:
# 根据union_indices 在 Points_WithH_Angle上确定极值点
test = ee.List(Points_WithH_Angle.get(0))
extracted_elements = unionFilter_indices.map(lambda index: ee.Dictionary(test.get(index)).set('index', index))
extracted_elements

## 利用rasterio保存栅格

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

# 准备数据
points = [feature['point_coordinates'] for feature in filtered_distortion_features]
values = [feature['values'] for feature in filtered_distortion_features]
types = [feature['distortion_type'] for feature in filtered_distortion_features]

# 确定栅格的范围和分辨率
x_min, y_min = min(p[0] for p in points), min(p[1] for p in points)
x_max, y_max = max(p[0] for p in points), max(p[1] for p in points)
resolution = 30  # 修改分辨率为 30

# 计算栅格的宽度和高度
width = int((x_max - x_min) / resolution) + 1
height = int((y_max - y_min) / resolution) + 1

# 创建栅格数组（每种畸变类型一个波段）
bands = {t: np.zeros((height, width), dtype=np.uint8) for t in set(types)}

# 填充栅格数组
for (x, y), value, t in zip(points, values, types):
    row = int((y_max - y) / resolution)
    col = int((x - x_min) / resolution)
    bands[t][row, col] = value

# 使用 rasterio 保存栅格文件
transform = rasterio.transform.Affine(30, 0, 202824.67259187606, 0, -30, 3324883.4699091096)
with rasterio.open(
    'output.tif', 'w', driver='GTiff',
    height=height, width=width,
    count=len(bands), dtype=np.uint8,
    crs = "+proj=utm +zone=46 +datum=WGS84 +units=m +no_defs", transform=transform
) as dst:
    for i, t in enumerate(sorted(bands.keys()), start=1):
        dst.write(bands[t], i)

## 数据绘图与保存

In [ ]:
import matplotlib.pyplot as plt

x_fit = np.array(x_fit)
y_fit = np.array(y_fit)
plt.figure(figsize=(20, 18))
plt.rcParams.update({'font.family': 'Times New Roman', 'font.size': 14, 'figure.dpi': 300})

plt.subplot(3, 3, 1)
plt.plot(x_fit, y_fit, color='red', linewidth=1.5)
plt.scatter(x_fit, y_fit, color='green', marker='o', s=20)
plt.ylabel('Y')
plt.title('Simulated Mountain Line')
plt.grid(True)

plt.subplot(3, 3, 2)
plt.plot(x_fit, first_derivative.getInfo(), color='green')
plt.ylabel('First Derivative')
plt.title('First Derivative the Simulated Mountain Line')
plt.grid(True)

plt.subplot(3, 3, 3)
plt.plot(x_fit, second_derivative.getInfo(), color='blue')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))  # 使用科学计数法表示y轴标注  
plt.ylabel('Second Derivative')
plt.title('Second Derivative of the Simulated Mountain Line')
plt.grid(True)

plt.subplot(3, 3, 4)
plt.plot(x_fit, third_derivative.getInfo(), color='purple')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))  # 使用科学计数法表示y轴标注
plt.xlabel('X')
plt.ylabel('Third Derivative')
plt.title('Third Derivative of the Simulated Mountain Line')
plt.grid(True)

plt.subplot(3, 3, 5)
plt.plot(x_fit, y_fit, color='red', linewidth=1.5)  # 先绘制曲线
plt.scatter(x_fit[intersection_indices.getInfo()], y_fit[intersection_indices.getInfo()], color='green', marker='o', s=30)  # 绘制转折点
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))  # 使用科学计数法表示y轴标注
plt.ylabel('Y')
plt.title('Selecting Points through Multi-level Derivatives')
plt.grid(True)

plt.subplot(3, 3, 6)
plt.plot(x_fit, curvature.getInfo(), color='darkslateblue', label='Curvature')  # 将曲线颜色改为青色
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))  # 使用科学计数法表示y轴标注
plt.ylabel('Curvature')
plt.title('Curvature of the Simulated Mountain Line')
plt.grid(True)

plt.subplot(3, 3, 9)
plt.plot(x_fit, y_fit, color='red')
plt.scatter(x_fit[unionFilter_indices.getInfo()], y_fit[unionFilter_indices.getInfo()], color='green', marker='o', s=30)
plt.ylabel('Y')
plt.title('Optimizing Point Selection through Curvature')
plt.grid(True)

plt.tight_layout()
plt.show()